# Absolute Zero at -272°C, says J. H. Lambert in 1777

Lambert, in his book, states measurements by a certain Mr. Amonton concerning the relation between temperature and pressure of air ("Pyrometrie" from 1777, page 40 [(archive.org link)](https://archive.org/details/bub_gb_G5I_AAAAcAAJ)). On page 47, he improves on the measurement of air pressure in an environment where water is boiling. By extrapolation, he re-calculates (and inproves upon) the temperature value of air when pressure approaches zero. Today's value of this absolute zero point is -273,15°C. His value, -272°C, is pretty close.

## Code Preamble

In [1]:
(require '[clojupyter.misc.helper :as helper])
(def _ (helper/add-dependencies '[org.mentat/emmy "0.32.0"]))
(def _ (helper/add-dependencies '[org.scicloj/kindly  "4-beta14"]))

#'user/_

In [2]:
(ns lambert
    (:refer-clojure :exclude [+ - * / zero? compare divide numerator denominator infinite? abs ref partial =])
    (:require [emmy.env :refer :all]
            [scicloj.kindly.v4.kind :as kind]
            [clojure.math :as math]))

nil

In [3]:
(defn n-transpose [m]
  (apply mapv vector m))

(defn tenpow [n]
  (apply * (repeat n 10)))

(defn round [x dec]
  (double (/ (math/round (* x (tenpow dec))) (tenpow dec))))

(defn tex [exp & [prefix]]
    (kind/tex (str "$" prefix (->TeX exp) "$")))

#'lambert/tex

## Data and Calculation

The book is in German. So, "Zolle" means inches, that is pressure; "Grade" means degrees, that is temperature in some old unit. He was kind enough to give verbal descriptions, "frierendes Wasser" is frozen water, i.e. our 0°C, and "siedendes Wasser" is boiling water, i.e. our 100°C.

In [4]:

(def desc {:zolle "Zolle des Amontonschen Thermometers"
           :grade "Grade das Luftthermometers"})

(def data
  {:zolle [51.6 54 54.4 58.2 58.9 55.9 59.9 61.10 64.2 67.3 58.5 73.0 54.0]
   :grade [1000 1049 1055 1129 1141 1082 1160 1201 1246 1306 1134 1417 1049]
   :C     ["frierendes Wasser"
           "Im Keller der Pariser Sternwarte"
           "Sternwarte2"
           "Wärem der Hand in mässiger Sommerluft, von verschiedenen Personen"
           "Handwärme 2"
           "Wärme der Luft, zur Zeit, da diese Wärme der Hände beobachtet worden"
           "schmelzende Butter"
           "schmelzend Unschlit"
           "gerinnend Wachs"
           "ein 10 bis 12 Gran schweres Stückchen Wachses ist ganz geschmolzen"
           "temperiert scheinendes Wasser"
           "siedendes Wasser"
           "temperierte Luft"]})

(def tb1 (n-transpose ((juxt :zolle :grade :C) data)))

(kind/table {:column-names ["Zolle | ", "Grade | ", "Descr."] 
             :row-vectors tb1})

51.6,1000,frierendes Wasser
54,1049,Im Keller der Pariser Sternwarte
54.4,1055,Sternwarte2
58.2,1129,"Wärem der Hand in mässiger Sommerluft, von verschiedenen Personen"
58.9,1141,Handwärme 2
55.9,1082,"Wärme der Luft, zur Zeit, da diese Wärme der Hände beobachtet worden"
59.9,1160,schmelzende Butter
61.1,1201,schmelzend Unschlit
64.2,1246,gerinnend Wachs
67.3,1306,ein 10 bis 12 Gran schweres Stückchen Wachses ist ganz geschmolzen
58.5,1134,temperiert scheinendes Wasser


Because we know that his 1000 degrees is our 0°C, we can covert his degrees to Celsius, given that we know the boiling point of water in his degrees.

In [5]:

(defn lc [b]
    (fn [g]
        (* (* 100 (- g 1000)) (/ 1 (- b 1000)))))

(defn luft->celsius [g bp]
  (round ((lc bp) g) 2))

(tex ((lc 'b) 'g) "t(g)=")

* t(g) ... temperature in degrees Celsius
* g ... degrees in Lambert's units
* b ... constant boiling point of water in Lambert's units

From Amonton's data, we know that the boiling point of water is 1417 and that in the basement of the Paris observatory, it had 1049 degrees. So we know the temperature in the basement in °C.

In [6]:

(tex (luft->celsius 1049 1417) "t_{1417}(1049)=")

Voilà, the full table:

In [7]:

(defn tb2 [sp]
  (mapv (fn[[z g d]] [z (luft->celsius g sp)]) tb1))

(kind/table {:column-names ["Zolle | ", "°C | "] 
             :row-vectors (tb2 1417)})

51.6,0.0
54,11.75
54.4,13.19
58.2,30.94
58.9,33.81
55.9,19.66
59.9,38.37
61.1,48.2
64.2,58.99
67.3,73.38
58.5,32.13


By exprapolating this table to Zolle=0 (i.e. linear regression), we calculate Amonton's absolute zero temperature to -241°C

In [8]:

(defn ordinate [xs ys]
  (let [n    (count xs)
        s_x  (reduce + xs)
        s_y  (reduce + ys)
        xy   (map #(* %1 %2) xs ys)
        s_xy (reduce + xy)
        x2   (map #(* % %) xs)
        s_x2 (reduce + x2)
        sx_2 (* s_x s_x)]
    (/ (- (* s_y s_x2) (* s_x s_xy))
       (- (* n s_x2) sx_2))))

(tex (round (apply ordinate (map (n-transpose (tb2 1417)) [0 1])) 1) "T_{1417}=")

On page 47, §89, Lambert states that his newly measured boiling point is 1370 (the old was 1417). "Siedepunkt Wasser 1370 bestimmt", "Ich werde als eine runde Zahl C = 1370 annehmen. Amontons fand 1417". With this, we can build the table above anew and extrapolate the according absolute zero temperature.

In [9]:

(tex (round (apply ordinate (map (n-transpose (tb2 1370)) [0 1])) 1) "T_{1370}=")

Let's calculate for $g=1360$ and $g=1380$ to estimate the measurement error of $T$.

In [10]:

(def bps [1360 1370 1380 1417])
(def Ts (mapv #(round % 1)
              (map #(apply ordinate
                           (map (n-transpose (tb2 %)) [0 1]))
           bps)))
(kind/table {:column-names ["b | ", "T"] 
             :row-vectors (n-transpose [bps Ts])})

1360,-279.4
1370,-271.8
1380,-264.7
1417,-241.2


So, based on his raw data, Lambert's zero temperature is -271,8°C. It is the extrapolation based on his 1370 degrees for the boiling point of water. If we assume a measurement error of +-10 digits, Lambert's T value is bound between -265°C and -280°C, indeed an improvement to Amonton.